In [1]:
import pandas as pd
from IPython.display import display, HTML

# We define the CSS that will make the table scrollable
# Definimos el CSS que hará que la tabla sea desplazable
css = """
.output {
    max-height: 500px; /* Adjust this to your needs / Ajusta esto a tus necesidades */
    overflow: scroll;  /* Allows scrolling / Permite desplazamiento */
}
"""
# We apply the CSS
# Aplicamos el CSS
HTML('<style>{}</style>'.format(css))

In [2]:
# Set the maximum number of rows to display
# Configuramos el máximo número de filas para mostrar
pd.set_option('display.max_rows', None)  # or a specific number of rows / o un número específico de filas

# Set the maximum number of columns to display
# Configuramos el máximo número de columnas para mostrar
pd.set_option('display.max_columns', None)  # or a specific number of columns / o un número específico de columnas

# Configure to wrap text for very long values
# Configuramos para que ajuste el texto a los valores muy largos
pd.set_option('display.max_colwidth', 90)

# Configurar pandas para mejorar la visualización de números: separador de miles
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
import pandas as pd

dtype = {
    'ACCOUNT_CDE': 'str',  
    'ASSETID':'str',
    'BUSINESS_UNIT':'str',
    'BUSINESS_UNIT_CDE': 'str', 
    'BUYER_ID': 'str', 
    'COMPANY_CDE': 'str',
    'COST_CENTER': 'str', 
    'COST_CENTER_DESC': 'str',
    'EXPENSE_REPORT_NAME': 'str',
    'FILE_NAME': 'str',
    'FISCAL_MONTH': 'str',
    'FISCAL_QUARTER': 'str',
    'FISCAL_YEAR': 'str',
    'INVOICE_COMMENT': 'str',
    'INVOICE_PMT_TERMS': 'str',
    'INVOICE_LINE_DESC': 'str',
    'INVOICE_LINE_NBR': 'str', 
    'INVOICE_LINE_QTY': 'str', 
    'INVOICE_LINE_UNIT_PRICE': 'str', 
    'INVOICE_NBR': 'str', 
    'INVOICE_PMT_TERMS_CDE': 'str', 
    'LEGAL_ENTITY_CDE': 'str',
    'LOCATION': 'str', 
    'LOCATION_ID': 'str',
    'PAYMENT_TYPE': 'str',
    'PO_LINE_DESC': 'str',
    'PO_LINE_ITEM_ID': 'str', 
    'PO_LINE_NBR': 'str', 
    'PO_LINE_QTY': 'str', 
    'PO_NBR': 'str', 
    'PROJECT_NAME':'str',
    'PROJECT_DESCRIPTION': 'str',
    'SUPPLIER_ADDRESS_1': 'str',
    'SUPPLIER_GRP': 'str',
    'SUPPLIER_ERP': 'str',
    'SUPPLIER_NBR': 'str',
    'SUPPLIER_NORMALIZED': 'str',
    'SUPPLIER_PMT_TERM_CDE': 'str',
    'SUPPLIER_STATE': 'str',
    'SUPPLIER_TAXID': 'str',
    'SUPPLIER_ZIP_POSTAL_CDE':'str',
    'DAYS_ATF_PO': 'str', 
    'INVOICE_LINE_AMOUNT': 'float',
    'EXCHANGE_RATE': 'float',
    'TOTAL_SPEND_USD': 'float',
    'TOTAL_SPEND':'float',
}

df = pd.read_csv('TBC_OUTPUT_CONCUR_FY23_MAR v.2.csv', encoding='UTF-8-SIG', encoding_errors='ignore', dtype=dtype)

df_shape = df.shape
print("DataFrame shape:", df_shape)

# Count of non-null values in 'SUPPLIER_NORMALIZED'
nonnull_count = df['SUPPLIER_NORMALIZED'].notna().sum()

print(f"Non-null values supplier normalized: {nonnull_count}")

DataFrame shape: (4002, 90)
Non-null values supplier normalized: 0


In [209]:


df = pd.read_csv('TBC_OUTPUT_CONCUR_FY23_MAR v.2.csv',
                 encoding='UTF-8-SIG',
                 encoding_errors='ignore',
                 usecols=['ASSETID', 'SUPPLIER_ERP', 'TOTAL_SPEND_USD']
                 )

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ASSETID          4002 non-null   object 
 1   SUPPLIER_ERP     3875 non-null   object 
 2   TOTAL_SPEND_USD  4002 non-null   float64
dtypes: float64(1), object(2)
memory usage: 93.9+ KB


In [210]:
#df.rename(columns={'Supplier - Normalized': 'SUPPLIER_NORMALIZED'}, inplace=True)
df.rename(columns={'TOTAL_SPEND_USD': 'TOTAL_SPEND'}, inplace=True)

In [44]:
# Suponiendo que TOTAL_SPEND podría tener espacios o caracteres no numéricos por error
# df['TOTAL_SPEND'] = df['TOTAL_SPEND'].replace(r'[^\d.]', '', regex=True).astype(float)

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ASSETID       4002 non-null   object 
 1   SUPPLIER_ERP  3875 non-null   object 
 2   TOTAL_SPEND   4002 non-null   float64
dtypes: float64(1), object(2)
memory usage: 93.9+ KB


# Exploration

In [212]:
# Crearemos un diccionario para almacenar los conteos de valores únicos de cada columna
conteos_unicos = {}

# Iteramos sobre cada columna en el DataFrame
for columna in df.columns:
    # Contamos los valores únicos en cada columna y los almacenamos en el diccionario
    conteos_unicos[columna] = df[columna].nunique()

# Mostramos el resultado
print(conteos_unicos)

{'ASSETID': 4002, 'SUPPLIER_ERP': 2168, 'TOTAL_SPEND': 2910}


In [213]:
import plotly.graph_objects as go
import plotly.express as px

# Reemplazar valores nulos en 'SUPPLIER_ERP' por 'Unknown' directamente en el DataFrame
df['SUPPLIER_ERP'] = df['SUPPLIER_ERP'].fillna('UNKNOWN')

# Agrupar el dataframe por 'SUPPLIER_ERP' y sumar 'TOTAL_SPEND'
df_grouped = df.groupby('SUPPLIER_ERP', as_index=False)['TOTAL_SPEND'].sum()

# Ordenar los valores sumados en orden descendente para obtener los proveedores con mayores gastos
df_grouped_sorted = df_grouped.sort_values(by='TOTAL_SPEND', ascending=False).head(40)

# Crear una escala de colores basada en los valores de 'TOTAL_SPEND'
color_scale = px.colors.sequential.Viridis

# Mapear el valor de 'TOTAL_SPEND' a la escala de colores
df_grouped_sorted['color'] = df_grouped_sorted['TOTAL_SPEND'].apply(lambda x: color_scale[int(((x - df_grouped_sorted['TOTAL_SPEND'].min()) / (df_grouped_sorted['TOTAL_SPEND'].max() - df_grouped_sorted['TOTAL_SPEND'].min())) * (len(color_scale) - 1))])

# Crear el gráfico de barras
fig = go.Figure(data=[go.Bar(
    x=df_grouped_sorted['SUPPLIER_ERP'],  # Nombres de los proveedores
    y=df_grouped_sorted['TOTAL_SPEND'],  # Gasto total sumado
    marker=dict(color=df_grouped_sorted['color']),  # Usar la escala de colores mapeada
    textposition='outside'
)])

# Actualizar el layout para añadir título y ajustar el eje Y
fig.update_layout(
    title_text='Top 40 Suppliers by Total Spend Aggregated',  # Título del gráfico
    xaxis_tickangle=-45,
    xaxis_title='Supplier',
    yaxis=dict(
        title='Total Spend ($)',
        tickprefix="$",
        tickformat=',.2f',
    ),
    template='plotly_white',
    autosize=True
)

# Ajustar el formato del texto para mostrar valores monetarios correctamente
fig.update_traces(texttemplate='%{text:$,.2f}')

# Mostrar el gráfico
fig.show()


In [214]:
import plotly.express as px

df['SUPPLIER_ERP_WONULL'] = df['SUPPLIER_ERP'].fillna('UNKNOWN')

# Contar las ocurrencias de cada valor en 'SUPPLIER_NORMALIZED'
value_counts = df['SUPPLIER_ERP_WONULL'].value_counts()

# Convertir el conteo en un DataFrame para visualización
counts_df = pd.DataFrame(value_counts).reset_index()
counts_df.columns = ['Supplier', 'Frequency']

# Crear un gráfico de barras con Plotly Express
fig = px.bar(counts_df.head(40), x='Supplier', y='Frequency',
             title="Top 40 Suppliers by Ocurrency",
             labels={'Supplier': 'Supplier Name', 'Frequency': 'Ocurrency'},
             color='Frequency',  # Color por frecuencia para una visualización más rica
             height=400)  # Ajustar la altura según necesidades

# Mostrar el gráfico
fig.show()

In [215]:
import plotly.graph_objects as go
from collections import Counter

# Asumiendo que 'df' es tu DataFrame y ya está cargado
# Contar las ocurrencias de cada palabra en la columna 'SUPPLIER_NORMALIZED'
word_counts = Counter(" ".join(df['SUPPLIER_ERP_WONULL']).split())

# Convertir el contador en un DataFrame
words_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency'])
words_df = words_df.sort_values(by='Frequency', ascending=False).head(200)

# Crear una tabla interactiva con Plotly Graph Objects
fig = go.Figure(data=[go.Table(
    header=dict(values=['Word', 'Frequency'],
                fill_color='paleturquoise',
                align='left',
                font=dict(size=14)),
    cells=dict(values=[words_df['Word'], words_df['Frequency']],
               fill_color='lavender',
               align='left'))
])

# Ajustar los márgenes de la figura y añadir un título
fig.update_layout(
    title_text="Most Frequent Words in Supplier Normalized",  # Título de la tabla
    title_x=0.5,  # Centrar el título
    title_font=dict(size=20),
    margin=dict(l=0, r=0, t=50, b=0),  # Ajustar márgenes para acomodar el título
    autosize=True
)

# Mostrar la figura
fig.show()

### Diccionario

In [216]:
# Agrupar el dataframe por 'SUPPLIER_NORMALIZED' y sumar 'TOTAL_SPEND'
df_grouped = df.groupby('SUPPLIER_ERP', as_index=False)['TOTAL_SPEND'].sum()

# Calcular el gasto total en todo el dataset
total_spend_all = df_grouped['TOTAL_SPEND'].sum()

# Lista de números de proveedores top para los cuales calcular el porcentaje
top_counts = [25, 50, 100, 200, 300, 500]

# Imprimir el porcentaje del total spend que cada grupo de proveedores top abarca
for count in top_counts:
    # Ordenar los valores sumados en orden descendente para obtener los proveedores con mayores gastos
    df_top = df_grouped.sort_values(by='TOTAL_SPEND', ascending=False).head(count)
    # Calcular el gasto total de los proveedores top seleccionados
    total_spend_top = df_top['TOTAL_SPEND'].sum()
    # Calcular el porcentaje del total que representa
    percentage_top = (total_spend_top / total_spend_all) * 100
    # Imprimir la información
    print(f"Seleccionando los {count} primeros proveedores, estos incluyen el {percentage_top:.2f}% del total spend en el dataset.")

Seleccionando los 25 primeros proveedores, estos incluyen el 47.99% del total spend en el dataset.
Seleccionando los 50 primeros proveedores, estos incluyen el 58.19% del total spend en el dataset.
Seleccionando los 100 primeros proveedores, estos incluyen el 67.88% del total spend en el dataset.
Seleccionando los 200 primeros proveedores, estos incluyen el 77.63% del total spend en el dataset.
Seleccionando los 300 primeros proveedores, estos incluyen el 82.63% del total spend en el dataset.
Seleccionando los 500 primeros proveedores, estos incluyen el 88.69% del total spend en el dataset.


In [51]:
# Extraemos el top whatever u want to provedores en terminos de total_spend para convertirlos en un diccionario OJO, REVISE QUE NO CAUSE ERRORES UN NOMBRE MUY CORTO, EJ: UBER, DELTA

# Reemplazar valores nulos en 'SUPPLIER_NORMALIZED' por 'Unknown'
# df['SUPPLIER_NORMALIZED'] = df['SUPPLIER_NORMALIZED'].fillna('Unknown') # no es necesario porque ya se corrio generando el grafico de total_spend por supplier

# Agrupar el dataframe por 'SUPPLIER_NORMALIZED' y sumar 'TOTAL_SPEND'
df_grouped = df.groupby('SUPPLIER_NORMALIZED', as_index=False)['TOTAL_SPEND'].sum()

# Ordenar los valores sumados en orden descendente para obtener los proveedores con mayores gastos
df_top = df_grouped.sort_values(by='TOTAL_SPEND', ascending=False).head(25) # AQUI selecciona el numero de proveedores que quieras usar para crear el diccionario

# Hacer una copia de la columna 'SUPPLIER_NORMALIZED'
df_top['if'] = df_top['SUPPLIER_NORMALIZED']

# Renombrar la columna original 'SUPPLIER_NORMALIZED' a 'then'
df_top.rename(columns={'SUPPLIER_NORMALIZED': 'then'}, inplace=True)

# Seleccionar solo las columnas renombradas para simplificar el DataFrame
df_dict = df_top[['if', 'then']]

df_dict


,if,then
4804,HILTON HOTELS,HILTON HOTELS
3009,DELTA AIRLINES,DELTA AIRLINES
583,AMERICAN AIRLINES,AMERICAN AIRLINES
4637,HAMPTON INNS,HAMPTON INNS
9295,SOUTHWEST AIRLINES,SOUTHWEST AIRLINES
4757,HERTZ,HERTZ
3233,DOUBLETREE HOTELS,DOUBLETREE HOTELS
8321,RESIDENCE INNS,RESIDENCE INNS
7109,NULL,NULL
6391,MARRIOTT HOTELS,MARRIOTT HOTELS


In [52]:
df_dict.to_excel(r'tbc_suppliers_dictionary_vv1_050624.xlsx', index=False)

In [217]:
# Carga el archivo Excel en un DataFrame
df_patterns = pd.read_excel(r'tbc_suppliers_dictionary_vv1_050624.xlsx')
df_patterns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   if contains  48 non-null     object
 1   then         48 non-null     object
dtypes: object(2)
memory usage: 900.0+ bytes


In [218]:
# Crearemos un diccionario para almacenar los conteos de valores únicos de cada columna
conteos_unicos = {}

# Iteramos sobre cada columna en el DataFrame
for columna in df_patterns.columns:
    # Contamos los valores únicos en cada columna y los almacenamos en el diccionario
    conteos_unicos[columna] = df_patterns[columna].nunique()

# Mostramos el resultado
print(conteos_unicos)

{'if contains': 48, 'then': 44}


In [219]:
# Convierte el DataFrame en un diccionario
patterns = pd.Series(df_patterns['then'].values, index=df_patterns['if contains']).to_dict()

# Imprimir la cantidad de elementos en el diccionario
print(f'El diccionario tiene {len(patterns)} elementos.')

El diccionario tiene 48 elementos.


In [220]:
# Imprimir los primeros 5 elementos del diccionario para verificar
for pattern, normalized_name in list(patterns.items())[:10]:
    print(f'Pattern: {pattern}, Normalized Name: {normalized_name}')

Pattern: HILTON, Normalized Name: HILTON HOTELS
Pattern: DELTA, Normalized Name: DELTA AIRLINES
Pattern: AMERICAN AIRLINES, Normalized Name: AMERICAN AIRLINES
Pattern: HAMPTON INN, Normalized Name: HAMPTON INNS
Pattern: SOUTHWES, Normalized Name: SOUTHWEST AIRLINES
Pattern: HERTZ, Normalized Name: HERTZ
Pattern: DOUBLETREE, Normalized Name: DOUBLETREE HOTELS
Pattern: RESIDENCE INNS, Normalized Name: RESIDENCE INNS
Pattern: MARRIOTT, Normalized Name: MARRIOTT HOTELS
Pattern: UNITED AIRLINES, Normalized Name: UNITED AIRLINES


In [221]:
patterns

{'HILTON': 'HILTON HOTELS',
 'DELTA': 'DELTA AIRLINES',
 'AMERICAN AIRLINES': 'AMERICAN AIRLINES',
 'HAMPTON INN': 'HAMPTON INNS',
 'SOUTHWES': 'SOUTHWEST AIRLINES',
 'HERTZ': 'HERTZ',
 'DOUBLETREE': 'DOUBLETREE HOTELS',
 'RESIDENCE INNS': 'RESIDENCE INNS',
 'MARRIOTT': 'MARRIOTT HOTELS',
 'UNITED AIRLINES': 'UNITED AIRLINES',
 'HOMES TO SUITES': 'HOMES TO SUITES',
 'EMBASSY SUITES': 'EMBASSY SUITES',
 'COURTYARD': 'COURTYARDS',
 'HOLIDAY INN': 'HOLIDAY INN',
 'HYATT': 'HYATT HOTELS',
 'TOOJAYS': 'TOOJAYS',
 'CHICK FIL A': 'CHICK FIL A',
 'IMDA': 'IMDA',
 'HOMEWOOD SUITES': 'HOMEWOOD SUITES',
 'AIR CAN': 'AIR CANADA',
 'OFFICE DEPOT': 'OFFICE DEPOT INC',
 'OFFICEMAX': 'OFFICE DEPOT INC',
 'FAIRFIELD INNS': 'FAIRFIELD INNS',
 'WALMART': 'WALMART',
 'WAL MART': 'WALMART',
 'STARBUCKS': 'STARBUCKS',
 'BUFFALO WILD': 'BUFFALO WILD WINGS',
 'DUNKIN': 'DUNKIN DONUTS',
 'FAIRFIELD INN': 'FAIRFIELD INNS',
 'FEDEX': 'FEDEX',
 'FED EX': 'FEDEX',
 'FIREHOUSE': 'FIREHOUSE SUBS',
 'JERSEY MIKES': '

# Cleaning

### Funcion y aplicacion de la funcion de Diana:

In [6]:
import re

def normalize_supplier_name(row):
    word = row['SUPPLIER_ERP']
    
    # If SUPPLIER_NORMALIZED is not blank, return its value without changes
    if not pd.isna(row['SUPPLIER_NORMALIZED']) and row['SUPPLIER_NORMALIZED']:
        return row['SUPPLIER_NORMALIZED']

    if pd.isna(word):
        return ''
    

    # Strip '#' and anything following it from right to left
    word =re.sub(r'#.*$', '', word)
    
    # Remove trailing whitespaces
    word = word.rstrip()

    # For concur (credit card) data only`
    payment_methods = ['TCB*','NIC*  ','CF* ','IN *','GLF*','WPY*','ZSK*','TLF*','TN*','PY *','PLACPAY*','L2G*','IC* ', 'B2P*', 'CCI*', 
                       'CEP*', 'CKE*', 'EIG*', 'FSP*', 'HAD*', 'PAR*', 'PAYPAL *', 'PMT*', 'PP*', 'SLICE*', 'SQ *', 'TST*',
                       '4TE*', 'UEP*', 'ORACL*']
    
    # Remove listed payment methods from supplier name
    for method in payment_methods:
        if word.startswith(method):
            word = word[len(method):].strip('*').strip()
    
    word = word.replace('$', '').replace('%', '').replace('^', '').replace('#', '').replace('(', '').replace(')', '')
    word = word.replace('[', '').replace(']', '').replace(';', '').replace(':', '').replace('"', '').replace("'", "")
    word = word.replace('"', '').replace(',', '').replace('’', '').replace('-', ' ').replace('/', ' ').replace('°', '')

    to_keep = ['.COM', '.com', '.ORG', '.org', '.NET', '.net', 'WWW.', 'www.']
    if any([keep in word for keep in to_keep]):
        return word
    else:
        word = word.replace('.', '')
        return word.upper()
    
    

In [7]:
# Apply the normalization function to create the SUPPLIER_NORMALIZED column
df['SUPPLIER_NORMALIZED'] = df.apply(normalize_supplier_name, axis=1)

# Print the distinct count of supplier erp and normalized to test re results of the normalization function
print(f"Distinct values before normalization: {df['SUPPLIER_ERP'].nunique()}")
print(f"Distinct values after normalization: {df['SUPPLIER_NORMALIZED'].nunique()}")

Distinct values before normalization: 2168
Distinct values after normalization: 1891


In [4]:
#Final export for Insigths
df.to_csv('TBC_OUTPUT_SAP_FY23_MAR v.2.csv', encoding='UTF-8-SIG', index=False)

In [8]:
# Clustering operation Concur data input

selected_columns =['FILE_NAME','ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED', 'TOTAL_SPEND_USD']
df_selected =df[selected_columns]

shape_before = df_selected.shape
print("Row count before grouping and with blanks:", shape_before)

df_summarized = df_selected.groupby(['FILE_NAME','ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'])['TOTAL_SPEND_USD'].sum().reset_index()

#Use .loc to strip whitespaces in 'SUPPLIER_ERP' and 'SUPPLIER_NORMALIZED'
df_summarized.loc[:, 'SUPPLIER_ERP'] = df_summarized['SUPPLIER_ERP'].str.strip()
df_summarized.loc[:, 'SUPPLIER_NORMALIZED'] = df_summarized['SUPPLIER_NORMALIZED'].str.strip()

concur_data = df_summarized.dropna(subset=['SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'], how='any', inplace=False)


shape_after= concur_data.shape
print("Row count after grouping and without blanks:", shape_after)

concur_data.to_csv('concur supplier list mar v.1.csv', encoding='UTF-8-SIG', index=False)

Row count before grouping and with blanks: (4002, 5)
Row count after grouping and without blanks: (3875, 5)


### Our code 

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ASSETID              4002 non-null   object 
 1   SUPPLIER_ERP         4002 non-null   object 
 2   TOTAL_SPEND          4002 non-null   float64
 3   SUPPLIER_ERP_WONULL  4002 non-null   object 
dtypes: float64(1), object(3)
memory usage: 125.2+ KB


In [223]:
# Crearemos un diccionario para almacenar los conteos de valores únicos de cada columna
conteos_unicos = {}

# Iteramos sobre cada columna en el DataFrame
for columna in df.columns:
    # Contamos los valores únicos en cada columna y los almacenamos en el diccionario
    conteos_unicos[columna] = df[columna].nunique()

# Mostramos el resultado
print(conteos_unicos)

{'ASSETID': 4002, 'SUPPLIER_ERP': 2169, 'TOTAL_SPEND': 2910, 'SUPPLIER_ERP_WONULL': 2169}


In [224]:
import re

def normalize_supplier_name(name, patterns):
    """
    Normalize supplier names based on given patterns.

    Args:
    name (str): The name of the supplier to normalize.
    patterns (dict): A dictionary of patterns where key is the part of the name to match and
                     the value is the normalized name.

    Returns:
    str: The normalized name if a pattern is matched, otherwise the original name.
    """
    
    # Comprobar si el nombre es una cadena
    if isinstance(name, str):
        # Convert the name to uppercase for case-insensitive comparison
        name_upper = name.upper()
        
        # Check each pattern in the dictionary
        for pattern, normalized_name in patterns.items():
            # If the pattern is found in the name, return the normalized name
            if re.search(pattern, name_upper):
                return normalized_name
    
    # If no pattern matches or if name is not a string, return the original name
    return name

# Example usage
example_names = [
    'SALTGRASS ARKANSAS INC',
    'WAL-MART SUPERCENTER 878',
    'CHICK-FIL-A 01393',
    'WHATABURGER346',
    'WHATABURGER RESTAURANTS LLC',
    'WHATABURGER INC 708',
    'CHICK FIL A 02045',
    'CHICK FIL A 01393',
    'WAL MART', 
    'WALMART21012',
    None,  # Agregar un ejemplo no string para probar
    12345  # Otro ejemplo no string
]

# Normalize the example names
normalized_names = [normalize_supplier_name(name, patterns) for name in example_names]
normalized_names

['SALTGRASS ARKANSAS INC',
 'WAL-MART SUPERCENTER 878',
 'CHICK-FIL-A 01393',
 'WHATABURGER346',
 'WHATABURGER RESTAURANTS LLC',
 'WHATABURGER INC 708',
 'CHICK FIL A',
 'CHICK FIL A',
 'WALMART',
 'WALMART',
 None,
 12345]

In [225]:
# Aplicar la función de normalización a la columna 'supplier_name'
df['SUPPLIER_CLEAN'] = df['SUPPLIER_ERP'].apply(lambda x: normalize_supplier_name(x, patterns))

unique_before = df['SUPPLIER_ERP'].nunique()
unique_after = df['SUPPLIER_CLEAN'].nunique()

# Imprimir el conteo distinto de valores antes y despues de la operacionn
print(f"Distinct values before operation: {unique_before}")
print(f"Distinct values after operation: {unique_after}")

# Calcular el numero de filas afectadas por la operacion 
values_normalized = unique_before - unique_after
print(f"Number of values normalized through the function: {values_normalized}")

# Crear un DataFrame con solo los registros afectados para comparar el antes y el después
affected_df = df[df['SUPPLIER_ERP'] != df['SUPPLIER_CLEAN']]

# Contar el numero de filas afectadas por la operacion
rows_affected = affected_df.shape[0]
print(f"Number of rows affected by the dictionary function: {rows_affected}")

# Mostrar el número de registros afectados y los detalles de los registros afectados
affected_df[['SUPPLIER_ERP', 'SUPPLIER_CLEAN']].sort_values(by='SUPPLIER_ERP', ascending=True)


Distinct values before operation: 2169
Distinct values after operation: 1710
Number of values normalized through the function: 459
Number of rows affected by the dictionary function: 1671


,SUPPLIER_ERP,SUPPLIER_CLEAN
3277,64199 - HILTON DOWNTOW,HILTON HOTELS
3631,A DUNKIN LAS,DUNKIN DONUTS
2933,AIR CAN 0144263076334,AIR CANADA
3132,AIR CAN 01480922553040,AIR CANADA
1501,AIR CAN* 00142191709599,AIR CANADA
2929,AIR CAN* 00144263551182,AIR CANADA
671,AIR CAN* 00144263591233,AIR CANADA
1596,American Airlines,AMERICAN AIRLINES
3438,American Airlines,AMERICAN AIRLINES
3425,American Airlines,AMERICAN AIRLINES


In [81]:
# df = df.drop(['SUPPLIER_ERP_HASH_TRIM'], axis=1)
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11203 entries, 0 to 11202
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   SUPPLIER_NORMALIZED  11203 non-null  object 
 1   TOTAL_SPEND          11203 non-null  float64
dtypes: float64(1), object(1)
memory usage: 175.2+ KB


### Funcion para remover el asterisco y lo que le sigue 

In [226]:
import re

def remove_hash_and_trim(word):
    # Eliminar todo después de '#'
    word = re.sub(r'#.*$', '', word)
    # Eliminar espacios finales
    word = word.rstrip()
    return word

# Supongamos que df es tu DataFrame y que 'SUPPLIER_CLEAN' es la columna relevante
# Aplicar la función a la columna 'SUPPLIER_CLEAN'
df['SUPPLIER_ERP_HASH_TRIM'] = df['SUPPLIER_CLEAN'].apply(remove_hash_and_trim)

unique_before = df['SUPPLIER_CLEAN'].nunique()
unique_after = df['SUPPLIER_ERP_HASH_TRIM'].nunique()

# Imprimir el conteo distinto de valores antes y despues de la operacionn
print(f"Distinct values before operation: {unique_before}")
print(f"Distinct values after operation: {unique_after}")

# Calcular el numero de filas afectadas por la operacion 
values_normalized = unique_before - unique_after
print(f"Number of values normalized through the function: {values_normalized}")

# Identificar las filas afectadas por la operacion 
changed_rows = df[df['SUPPLIER_CLEAN'] != df['SUPPLIER_ERP_HASH_TRIM']]

# Contar el numero de filas afectadas por la operacion
rows_affected = changed_rows.shape[0]
print(f"Number of rows affected by the hash and trim function: {rows_affected}")

# Mostrar filas afectadas para revision
changed_rows[['SUPPLIER_CLEAN', 'SUPPLIER_ERP_HASH_TRIM']]


Distinct values before operation: 1710
Distinct values after operation: 1544
Number of values normalized through the function: 166
Number of rows affected by the hash and trim function: 324


,SUPPLIER_CLEAN,SUPPLIER_ERP_HASH_TRIM
14,WAL-MART #1466,WAL-MART
127,TIM HORTONS #915678,TIM HORTONS
131,PAPA JOHNS #744,PAPA JOHNS
154,ZAXBY'S #74345,ZAXBY'S
155,BP#1309600LOVE PETRQPS,BP
159,BUC-EE'S #57,BUC-EE'S
161,CASEY S # 4434,CASEY S
183,PANDA EXPRESS #2079,PANDA EXPRESS
185,PANERA BREAD #204993 P,PANERA BREAD
186,EXXON KENJO MARKET #45,EXXON KENJO MARKET


### Funcion para remover metodos de pago

In [227]:
def remove_payment_methods(word):
    payment_methods = ['TCB*', 'NIC*  ', 'CF* ', 'IN *', 'GLF*', 'WPY*', 'ZSK*', 'TLF*', 'TN*', 'PY *', 'PLACPAY*',
                       'L2G*', 'IC* ', 'B2P*', 'CCI*', 'CEP*', 'CKE*', 'EIG*', 'FSP*', 'HAD*', 'PAR*', 'PAYPAL *', 
                       'PMT*', 'PP*', 'SLICE*', 'SQ *', 'TST*', '4TE*', 'UEP*', 'ORACL*']
    
    for method in payment_methods:
        if word.startswith(method.strip()):
            word = word[len(method.strip()):].strip('*').strip()
    return word

# Aplicar la función remove_payment_methods a la columna 'SUPPLIER_ERP_HASH_TRIM'
df['SUPPLIER_ERP_REM_PAYME'] = df['SUPPLIER_ERP_HASH_TRIM'].apply(remove_payment_methods)

# Contar valores únicos antes y después de la operación
unique_before = df['SUPPLIER_ERP_HASH_TRIM'].nunique()
unique_after = df['SUPPLIER_ERP_REM_PAYME'].nunique()

# Imprimir el conteo distinto de valores antes y después de la operación
print(f"Distinct values before operation: {unique_before}")
print(f"Distinct values after operation: {unique_after}")

# Calcular el número de valores afectados por la operación
values_normalized = unique_before - unique_after
print(f"Number of values normalized through the function: {values_normalized}")

# Identificar las filas afectadas por la operación
changed_rows = df[df['SUPPLIER_ERP_HASH_TRIM'] != df['SUPPLIER_ERP_REM_PAYME']]

# Contar el número de filas afectadas por la operación
rows_affected = changed_rows.shape[0]
print(f"Number of rows affected by the remove payment methods function: {rows_affected}")

# Opcionalmente, mostrar las filas afectadas para revisión
changed_rows[['SUPPLIER_ERP_HASH_TRIM', 'SUPPLIER_ERP_REM_PAYME']]

Distinct values before operation: 1544
Distinct values after operation: 1543
Number of values normalized through the function: 1
Number of rows affected by the remove payment methods function: 218


,SUPPLIER_ERP_HASH_TRIM,SUPPLIER_ERP_REM_PAYME
42,TST* TRILL BURGER,TRILL BURGER
46,TST* SOYA 2,SOYA 2
151,TST* MI AMAICHE,MI AMAICHE
153,TST* MIDWOOD SMOKE HOU,MIDWOOD SMOKE HOU
193,TST* MCGHIN'S SOUTHERN,MCGHIN'S SOUTHERN
263,SQ *QUICK TRIGGER BREW,QUICK TRIGGER BREW
264,SQ *WILL-O-WISP CAFE,WILL-O-WISP CAFE
287,TST* MERCATO KITCHEN-,MERCATO KITCHEN-
288,TST* MERCATO KITCHEN-,MERCATO KITCHEN-
304,TST* SALT AND SMOKE -,SALT AND SMOKE -


### Funcion para caracteres especiales

In [228]:
def finalize_name_cleaning(word):
    # Reemplazar caracteres específicos
    chars_to_remove = ['$', '%', '^', '#', '(', ')', '[', ']', ';', ':', '"', "'", ',', '’', '-', '/', '°']
    for char in chars_to_remove:
        word = word.replace(char, ' ')

    # Reemplazar espacios adicionales por un solo espacio
    word = ' '.join(word.split())

    # Chequear si el nombre contiene dominios importantes
    to_keep = ['.COM', '.com', '.ORG', '.org', '.NET', '.net', 'WWW.', 'www.']
    if any(keep in word for keep in to_keep):
        return word
    else:
        # Eliminar puntos y convertir a mayúsculas si no contiene dominios especiales
        word = word.replace('.', '').upper()

    return word

# Ejemplo de aplicación de la función
df['SUPPLIER_CLEANED_FINAL'] = df['SUPPLIER_ERP_REM_PAYME'].apply(finalize_name_cleaning)

# Contar valores únicos antes y después de la limpieza final
unique_before_final = df['SUPPLIER_ERP_REM_PAYME'].nunique()
unique_after_final = df['SUPPLIER_CLEANED_FINAL'].nunique()

print(f"Distinct values before final cleaning: {unique_before_final}")
print(f"Distinct values after final cleaning: {unique_after_final}")

# Calcular el número de valores unificados por la función de limpieza final
values_unified = unique_before_final - unique_after_final
print(f"Number of values unified through the final cleaning function: {values_unified}")

# Identificar las filas afectadas por la limpieza final
changed_rows_final = df[df['SUPPLIER_ERP_REM_PAYME'] != df['SUPPLIER_CLEANED_FINAL']]

# Contar el número de filas afectadas por la limpieza final
rows_affected_final = changed_rows_final.shape[0]
print(f"Number of rows affected by the final cleaning function: {rows_affected_final}")

# Mostrar las filas afectadas para revisión (opcional)
changed_rows_final[['SUPPLIER_ERP_REM_PAYME', 'SUPPLIER_CLEANED_FINAL']]

Distinct values before final cleaning: 1543
Distinct values after final cleaning: 1516
Number of values unified through the final cleaning function: 27
Number of rows affected by the final cleaning function: 692


,SUPPLIER_ERP_REM_PAYME,SUPPLIER_CLEANED_FINAL
4,EATZI'S,EATZI S
7,BNA JIMMY JOHN'S,BNA JIMMY JOHN S
9,Pitt Stop,PITT STOP
14,WAL-MART,WAL MART
33,Microsoft,MICROSOFT
38,PLAYITA'S RESTAURANTE,PLAYITA S RESTAURANTE
39,LOS COMPADRES MEXICAN-,LOS COMPADRES MEXICAN
40,DD *STORECHICK-FIL-A,DD *STORECHICK FIL A
43,CITY CREEK BURGER CO.,CITY CREEK BURGER CO
44,Sunbizz,SUNBIZZ


### Funcion para remover numeros a la derecha

In [229]:
def remove_trailing_numbers(text):
    # Utiliza una expresión regular para eliminar números al final de la cadena
    return re.sub(r'\d+$', '', text).strip()

# Supongamos que df es tu DataFrame y que ya tiene una columna 'SUPPLIER_CLEANED_FINAL'
# que deseamos limpiar más.
df['SUPPLIER_NO_TRAIL_NUM'] = df['SUPPLIER_CLEANED_FINAL'].apply(remove_trailing_numbers)

# Contar valores únicos antes y después de la eliminación de números al final
unique_before = df['SUPPLIER_CLEANED_FINAL'].nunique()
unique_after = df['SUPPLIER_NO_TRAIL_NUM'].nunique()

# Imprimir el conteo de valores únicos
print(f"Distinct values before removing trailing numbers: {unique_before}")
print(f"Distinct values after removing trailing numbers: {unique_after}")

# Calcular el número de valores unificados por la función
values_unified = unique_before - unique_after
print(f"Number of values unified by removing trailing numbers: {values_unified}")

# Identificar las filas afectadas por la función
changed_rows = df[df['SUPPLIER_CLEANED_FINAL'] != df['SUPPLIER_NO_TRAIL_NUM']]

# Contar el número de filas afectadas por la función
rows_affected = changed_rows.shape[0]
print(f"Number of rows affected by removing trailing numbers: {rows_affected}")

# Mostrar las filas afectadas para revisión (opcional)
changed_rows[['SUPPLIER_CLEANED_FINAL', 'SUPPLIER_NO_TRAIL_NUM']]


Distinct values before removing trailing numbers: 1516
Distinct values after removing trailing numbers: 1304
Number of values unified by removing trailing numbers: 212
Number of rows affected by removing trailing numbers: 442


,SUPPLIER_CLEANED_FINAL,SUPPLIER_NO_TRAIL_NUM
0,AMERICAN 00180953551262,AMERICAN
10,THE UPS STORE 7592,THE UPS STORE
11,JETBLUE 27970588201065,JETBLUE
15,AMERICAN 0014430352586,AMERICAN
18,AMERICAN 0014430554307,AMERICAN
20,WHERE TRAVELER ST2859,WHERE TRAVELER ST
21,CAPITOL FILE TEMP 553,CAPITOL FILE TEMP
37,FIVE BELOW 1152,FIVE BELOW
46,SOYA 2,SOYA
130,WHITE CASTLE 050039,WHITE CASTLE


In [230]:
# Crearemos un diccionario para almacenar los conteos de valores únicos de cada columna
conteos_unicos = {}

# Iteramos sobre cada columna en el DataFrame
for columna in df.columns:
    # Contamos los valores únicos en cada columna y los almacenamos en el diccionario
    conteos_unicos[columna] = df[columna].nunique()

# Mostramos el resultado
print(conteos_unicos)

{'ASSETID': 4002, 'SUPPLIER_ERP': 2169, 'TOTAL_SPEND': 2910, 'SUPPLIER_ERP_WONULL': 2169, 'SUPPLIER_CLEAN': 1710, 'SUPPLIER_ERP_HASH_TRIM': 1544, 'SUPPLIER_ERP_REM_PAYME': 1543, 'SUPPLIER_CLEANED_FINAL': 1516, 'SUPPLIER_NO_TRAIL_NUM': 1304}


In [232]:
df.rename(columns={'SUPPLIER_NO_TRAIL_NUM': 'SUPPLIER_NORMALIZED'}, inplace=True)
#df.rename(columns={'  Sum(Total Spend USD) FY_2023  ': 'TOTAL_SPEND'}, inplace=True)

In [231]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4002 entries, 0 to 4001
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ASSETID                 4002 non-null   object 
 1   SUPPLIER_ERP            4002 non-null   object 
 2   TOTAL_SPEND             4002 non-null   float64
 3   SUPPLIER_ERP_WONULL     4002 non-null   object 
 4   SUPPLIER_CLEAN          4002 non-null   object 
 5   SUPPLIER_ERP_HASH_TRIM  4002 non-null   object 
 6   SUPPLIER_ERP_REM_PAYME  4002 non-null   object 
 7   SUPPLIER_CLEANED_FINAL  4002 non-null   object 
 8   SUPPLIER_NO_TRAIL_NUM   4002 non-null   object 
dtypes: float64(1), object(8)
memory usage: 281.5+ KB


In [233]:
selected_columns =['ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED', 'TOTAL_SPEND']
df_selected =df[selected_columns]

shape_before = df_selected.shape
print("Row count before grouping and with blanks:", shape_before)

df_selected = df_selected.groupby(['ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'])['TOTAL_SPEND'].sum().reset_index()

#Use .loc to strip whitespaces in 'SUPPLIER_ERP' and 'SUPPLIER_NORMALIZED'
df_selected.loc[:, 'SUPPLIER_ERP'] = df_selected['SUPPLIER_ERP'].str.strip()
df_selected.loc[:, 'SUPPLIER_NORMALIZED'] = df_selected['SUPPLIER_NORMALIZED'].str.strip()

# concur_data = df_summarized.dropna(subset=['SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'], how='any', inplace=False)
# Encontrar índices donde la columna 'SUPPLIER_NORMALIZED' es 'UNKNOWN'
indices_to_drop = df_selected[df_selected['SUPPLIER_NORMALIZED'] == 'UNKNOWN'].index
# Eliminar estas filas del DataFrame
concur_data = df_selected.drop(indices_to_drop)

shape_after= concur_data.shape
print("Row count after grouping and without blanks:", shape_after)

Row count before grouping and with blanks: (4002, 4)
Row count after grouping and without blanks: (3875, 4)


In [207]:
# Crearemos un diccionario para almacenar los conteos de valores únicos de cada columna
conteos_unicos = {}

# Iteramos sobre cada columna en el DataFrame
for columna in concur_data.columns:
    # Contamos los valores únicos en cada columna y los almacenamos en el diccionario
    conteos_unicos[columna] = concur_data[columna].nunique()

# Mostramos el resultado
print(conteos_unicos)

{'ASSETID': 3875, 'SUPPLIER_ERP': 2168, 'SUPPLIER_NORMALIZED': 1407, 'TOTAL_SPEND': 2870}


In [164]:
selected_columns_2 =['FILE_NAME','ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED_2', 'TOTAL_SPEND']
df_selected_2 =df[selected_columns]

shape_before = df_selected_2.shape
print("Row count before grouping and with blanks:", shape_before)

df_selected_2 = df_selected_2.groupby(['FILE_NAME','ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'])['TOTAL_SPEND'].sum().reset_index()

#Use .loc to strip whitespaces in 'SUPPLIER_ERP' and 'SUPPLIER_NORMALIZED'
df_selected_2.loc[:, 'SUPPLIER_ERP'] = df_selected_2['SUPPLIER_ERP'].str.strip()
df_selected_2.loc[:, 'SUPPLIER_NORMALIZED'] = df_selected_2['SUPPLIER_NORMALIZED'].str.strip()

# concur_data = df_summarized.dropna(subset=['SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'], how='any', inplace=False)
# Encontrar índices donde la columna 'SUPPLIER_NORMALIZED' es 'UNKNOWN'
indices_to_drop = df_selected_2[df_selected_2['SUPPLIER_NORMALIZED'] == 'UNKNOWN'].index
# Eliminar estas filas del DataFrame
concur_data_2 = df_selected_2.drop(indices_to_drop)

shape_after= concur_data_2.shape
print("Row count after grouping and without blanks:", shape_after)

Row count before grouping and with blanks: (51234, 5)
Row count after grouping and without blanks: (49417, 5)


In [234]:
concur_data.to_csv('concur supplier list mar v.6.csv', encoding='UTF-8-SIG', index=False)

In [149]:
# Clustering operation Concur data input

selected_columns =['FILE_NAME','ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED', 'TOTAL_SPEND', 'SUPPLIER_NORMALIZED_2']
df_selected =df[selected_columns]

shape_before = df_selected.shape
print("Row count before grouping and with blanks:", shape_before)

df_summarized = df_selected.groupby(['FILE_NAME','ASSETID','SUPPLIER_ERP', 'SUPPLIER_NORMALIZED', 'SUPPLIER_NORMALIZED_2'])['TOTAL_SPEND'].sum().reset_index()

#Use .loc to strip whitespaces in 'SUPPLIER_ERP' and 'SUPPLIER_NORMALIZED'
df_summarized.loc[:, 'SUPPLIER_ERP'] = df_summarized['SUPPLIER_ERP'].str.strip()
df_summarized.loc[:, 'SUPPLIER_NORMALIZED'] = df_summarized['SUPPLIER_NORMALIZED'].str.strip()
df_summarized.loc[:, 'SUPPLIER_NORMALIZED_2'] = df_summarized['SUPPLIER_NORMALIZED_2'].str.strip()


concur_data = df_summarized.dropna(subset=['SUPPLIER_ERP', 'SUPPLIER_NORMALIZED'], how='any', inplace=False)

shape_after= concur_data.shape
print("Row count after grouping and without blanks:", shape_after)

concur_data.to_csv('concur supplier list mar v.2.csv', encoding='UTF-8-SIG', index=False)

Row count before grouping and with blanks: (51234, 6)
Row count after grouping and without blanks: (51228, 11)


| Column                   | Distinct Values | Relative Variation | Percentage Variation   | Procedure Description                                        |
|--------------------------|-----------------|--------------------|------------------------|--------------------------------------------------------------|
| SUPPLIER_NORMALIZED      | 11203           | -                  | -                      | Data normalized from oracle                                  |
| SUPPLIER_CLEAN           | 11138           | -65                | -0.58%                 | Apply dictionary and null values handled as unknown          |
| SUPPLIER_ERP_HASH_TRIM   | 11035           | -103               | -0.92%                 | Removed everything to the right of a hash (#) symbol.        |
| SUPPLIER_ERP_REM_PAYME   | 11028           | -7                 | -0.063%                | 'Payment' related terms removed from names.                  |
| SUPPLIER_CLEANED_FINAL   | 10957           | -71                | -0.64%                 | Cleaned special characters while preserving internet domains.|
| SUPPLIER_NO_TRAIL_NUM    | 10039           | -918               | -8.37%                 | Trailing numbers removed from supplier names.                |



| Casos para considerar en las funciones | Obs |
|-|-|
| LAX - TERMINAL 3 |